In [ ]:
import torch
import pandas as pd
from tqdm import tqdm as tm
from src import Simulation as sim
from src import Dir_Reg as DR
from src import ABC_Reg
from src import Align

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("mps") if torch.backends.mps.is_available() else device

print(device)

cuda


In [2]:
# we want to investigate what happens to our estimation, when we use the correct model, but misspecify the dimension. 
# 1. generate some synthetic data with 8 dimensions
# 2. first do full oracle with the correct dimension to get base line
# 3. then embed the graph in lower dimension, run regression, and then compare result to full oracle

In [5]:
p, K, T = 6, 6, 2
# alpha_0 = torch.ones(K, p)*2
alpha_0 = torch.eye(p)*9 + 1
# alpha_0 = torch.kron(torch.eye(int(p/2))*9 + 1, torch.ones(2))

model = sim.ABC(time = T,
        nodes = 30,
        beta = [1, 1, -4, 5],
        alpha_0 = alpha_0)

seed_list = list(range(50))
n_list = list(range(1500, 6001, 1500))
p0_list = list(range(2, 7))

# Initialize an empty list to store results
results = []
for seed in seed_list:
    torch.manual_seed(seed)
    for n in n_list:
        for p0 in p0_list:
            # Update model settings
            model.update_settings(nodes = n)
            # Initialize estimation
            estimate = ABC_Reg.est(two_lat_pos = model.synth_data['lat_pos'],
                    two_adj_mat = model.synth_data['obs_adj'],
                    groups = K,
                    )
            # Perform estimation by specifying mode and embedding dimension p0
            estimate.specify_mode('NO', fit = True, embed_dim = p0)
            
            # Compute beta_est and info_lost
            beta_est = DR.fit.proj_beta(estimate.fitted.est_result["estimate"], DR.fit.gen_constraint(p0+1, True)).tolist()
            info_lost = estimate.fitted.est_result["info_lost"]
            
            # Create a dictionary for the current iteration
            result = {
                'seed': seed,
                'n': n,
                'p0': p0,
                'beta1': beta_est[0],
                'beta2': beta_est[1],
                'beta3': beta_est[2],
                'beta4': beta_est[3],
                'info_lost': info_lost  # Optional: Include if you want to store this value
            }
            # Append the result to the list
            results.append(result)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(results)
df.to_csv(r"simulated_data\dimension_robustness\robustness.csv")  

0.0
0.02
0.03
0.05
0.07
0.08
0.1
0.12
0.13
0.15
0.17
0.18
0.2
0.22
0.23
0.25
0.27
0.28
0.3
0.32
0.33
0.35
0.37
0.38
0.4
0.42
0.43
0.45
0.47
0.48
0.5
0.52
0.53
0.55
0.57
0.58
0.6
0.62
0.63
0.65
0.67
0.68
0.7
0.72
0.73
0.75
0.77
0.78
0.8
0.82


In [3]:
torch.manual_seed(5)

p, K, T, n = 6, 3, 2, 6000
alpha_0 = torch.ones(K, p)*2
# alpha_0 = torch.eye(p)*9 + 1

model = sim.ABC(time = T,
                nodes = n,
                beta = [1, 1, -4, 5],
                alpha_0 = alpha_0)

In [4]:
temp = ABC_Reg.est(two_lat_pos = model.synth_data['lat_pos'],
                   two_adj_mat = model.synth_data['obs_adj'],
                   groups = K,
                   )

In [5]:
temp.specify_mode('OL', fit = True)
DR.fit.proj_beta(temp.fitted.est_result["estimate"], DR.fit.gen_constraint(p, True))

tensor([ 0.9733,  0.8217, -4.2562,  4.3985])

In [6]:
p_0 = 3
temp.specify_mode('NO', fit = True, embed_dim = p_0)
print(temp.fitted.est_result["info_lost"])
DR.fit.proj_beta(temp.fitted.est_result["estimate"], DR.fit.gen_constraint(p_0+1, True))

0.01733333333333331


tensor([ 0.4031,  0.3385, -1.7095,  0.5608])

In [7]:
p_0 = 4
temp.specify_mode('NO', fit = True, embed_dim = p_0)
print(temp.fitted.est_result["info_lost"])
DR.fit.proj_beta(temp.fitted.est_result["estimate"], DR.fit.gen_constraint(p_0+1, True))

0.09533333333333338


tensor([ 0.3947,  0.0186, -1.5651,  0.8212])

In [8]:
p_0 = 5
temp.specify_mode('NO', fit = True, embed_dim = p_0)
print(temp.fitted.est_result["info_lost"])
DR.fit.proj_beta(temp.fitted.est_result["estimate"], DR.fit.gen_constraint(p_0+1, True))


0.2613333333333333


tensor([ 0.4584,  0.2393, -2.2085, -3.3477])

In [9]:
p_0 = 6
temp.specify_mode('NO', fit = True, embed_dim = p_0)
print(temp.fitted.est_result["info_lost"])
DR.fit.proj_beta(temp.fitted.est_result["estimate"], DR.fit.gen_constraint(p_0+1, True))

0.43066666666666664


tensor([ 0.6828,  0.8133, -3.9691, -3.2112])

In [10]:
p_0 = 7
temp.specify_mode('NO', fit = True, embed_dim = p_0)
print(temp.fitted.est_result["info_lost"])
DR.fit.proj_beta(temp.fitted.est_result["estimate"], DR.fit.gen_constraint(p_0+1, True))

0.7296666666666667


tensor([ 0.9345,  1.1643, -3.5133,  5.0706])